# Relabelling USA spending field to integers in order to run Spark

In [1]:
import google.datalab.bigquery as bq

In [3]:
%%bq query
select
mod_agency,
vendorname,
count(*) as count
from `fiery-set-171213.vrec.usa_spending_all`
where vendorcountrycode in ('UNITED STATES', 'USA: UNITED STATES OF AMERICA')
and contractingofficerbusinesssizedetermination in ('O: OTHER THAN SMALL BUSINESS', 'S: SMALL BUSINESS')
and mod_agency not in ("")
group by 1,2,3
order by count DESC
limit 100

contractingofficerbusinesssizedetermination,mod_agency,vendorname,count
O: OTHER THAN SMALL BUSINESS,97AS: DEFENSE LOGISTICS AGENCY,AMERISOURCEBERGEN DRUG CORPORA,676166
O: OTHER THAN SMALL BUSINESS,97AS: DEFENSE LOGISTICS AGENCY,"CARDINAL HEALTH 200, LLC",419654
O: OTHER THAN SMALL BUSINESS,97AS: DEFENSE LOGISTICS AGENCY,"CARDINAL HEALTH, INC",318861
O: OTHER THAN SMALL BUSINESS,4730: FEDERAL ACQUISITION SERVICE,NATIONAL INDUSTRIES FOR THE BLIND,308991
O: OTHER THAN SMALL BUSINESS,97AS: DEFENSE LOGISTICS AGENCY,"OWENS & MINOR DISTRIBUTION, INC.",262205
O: OTHER THAN SMALL BUSINESS,97AS: DEFENSE LOGISTICS AGENCY,AMERISOURCEBERGEN DRUG CORPORATION,243671
O: OTHER THAN SMALL BUSINESS,"1900: STATE, DEPARTMENT OF",MISCELLANEOUS FOREIGN CONTRACTORS,237245
O: OTHER THAN SMALL BUSINESS,97AS: DEFENSE LOGISTICS AGENCY,SCIENCE APPLICATIONS INTERNATIONAL CORPORATION,218120
O: OTHER THAN SMALL BUSINESS,"1900: STATE, DEPARTMENT OF",MISCELLANEOUS FOREIGN AWARDEES,217609
O: OTHER THAN SMALL BUSINESS,4730: FEDERAL ACQUISITION SERVICE,J & L AMERICA INC,210475


## usa_mod_agency_relabel

In [11]:
%%bq query
select
mod_agency,
ROW_NUMBER() OVER (ORDER BY user DESC) row_num
from
  (select
  mod_agency
  #REGEXP_REPLACE(mod_agency, r',', '') as user
  from `fiery-set-171213.vrec.usa_spending_all`
  group by 1)

user,user_row_num
9568: BROADCASTING BOARD OF GOVERNORS,34
2024: FEDERAL LAW ENFORCEMENT TRAINING CENTER,172
"1438: OFFICE OF SURFACE MINING, RECLAMATION AND ENFORCEMENT",220
9514: OCCUPATIONAL SAFETY AND HEALTH REVIEW COMMISSION,49
6930: FEDERAL RAILROAD ADMINISTRATION,115
1650: OCCUPATIONAL SAFETY AND HEALTH ADMINISTRATION,184
1645: MINE SAFETY AND HEALTH ADMINISTRATION,185
1630: EMPLOYMENT AND TRAINING ADMINISTRATION,188
1425: BUREAU OF RECLAMATION,226
1153: UNITED STATES TRADE AND DEVELOPMENT AGENCY,278


## usa_vendorname_relabel

In [13]:
%%bq query
select
vendorname,
ROW_NUMBER() OVER (ORDER BY vendorname DESC) row_num
from
  (select
  vendorname
  from `fiery-set-171213.vrec.usa_spending_all`
  group by 1)

item,item_row_num
orthopedic institute,34
ZYKRONIX INCORPORATED,172
ZWICK WINDOW SHADE COMPANY,220
"ZURIAN, STEVEN",297
ZUHEIR INCORPORATED,403
"ZREYATECH, LLC",491
"ZORZA, INC.",527
ZORN COMPRESSOR AND EQUIPMENT INCORPORATED,535
ZOMEGA TECHNOLOGY CORPORATION,660
"ZOLLO CFP, KIMBERLEY FEENEY",678


# Get relabeled table 

In [23]:
%%bq query
select
b.row_num as user_number,
c.row_num as item_number,
#vendorname as item,
#a.mod_agency as user,
count(*) as count
from `fiery-set-171213.vrec.usa_spending_all` as a
join `fiery-set-171213.vrec.usa_mod_agency_relabel` as b
on a.mod_agency = b.mod_agency
join `fiery-set-171213.vrec.usa_vendorname_relabel` as c
on a.vendorname = c.vendorname
where vendorcountrycode in ('UNITED STATES', 'USA: UNITED STATES OF AMERICA')
and contractingofficerbusinesssizedetermination in ('O: OTHER THAN SMALL BUSINESS', 'S: SMALL BUSINESS')
and a.mod_agency not in ("")
group by 1,2
order by count DESC
limit 10000

user_number,item_number,count
16,895988,676166
143,348288,469971
16,795270,419655
16,795221,318862
16,306804,262224
16,895987,243671
178,370992,239677
16,190617,218145
178,370994,217610
143,506505,210493
